In [133]:
import numpy as np
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp

def load_csv(filename):
    dataset = list()
    contains_missing_values = False
    
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            contains_missing_values = False 
            # Brisanje redova koji imaju ?. 
            for i in range(len(row)):
                if row[i] is '?':
                    contains_missing_values = True
                    break
            if contains_missing_values == False:
                dataset.append(row)

    return dataset

def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column])

def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

def dataset_minmax(dataset):
    minmax = list()
    stats = [[min(column), max(column)] for column in zip(*dataset)]
    return stats

def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)-1):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

def feedforward_initialize(n_inputs, n_hidden_nodes, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden_nodes)]
    output_layer = [{'weights':[random() for i in range(n_hidden_nodes + 1)]} for i in range(n_outputs)]
    network.append(hidden_layer)
    network.append(output_layer)
    return network

def train_test_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

def calculate_accuracy(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * neuron['output'] * (1 - neuron['output'])

def update_weights(network, row, learn_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += learn_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += learn_rate * neuron['delta']

def train_network(network, train, learn_rate, n_epochs, n_outputs):
    for epoch in range(n_epochs):
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            backward_propagate_error(network, expected)
            update_weights(network, row, learn_rate)

def back_propagation(train, test, learn_rate, n_epochs, n_hidden_nodes):
    n_inputs = len(train[0]) - 1
    n_outputs = len(set([row[-1] for row in train]))
    network = feedforward_initialize(n_inputs, n_hidden_nodes, n_outputs)
    train_network(network, train, learn_rate, n_epochs, n_outputs)
    predictions = list()
    for row in test:
        prediction = predict(network, row)
        predictions.append(prediction)
    return(predictions)

def feedforward_algorithm(dataset, n_folds, learn_rate, n_epochs, n_hidden_nodes):
    folds = train_test_split(dataset, n_folds)  
    scores = list()
    
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = back_propagation(train_set, test_set, learn_rate, n_epochs, n_hidden_nodes)
        actual = [row[-1] for row in fold]
        accuracy = calculate_accuracy(actual, predicted)
        
        scores.append(accuracy)
    return scores

# Inicijalizacija elm neuronske mreze.
def elm_initialize(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs)]} for i in range(n_hidden)]
    bias = [{"bias": [random() for i in range(n_hidden)]}]
    network.append(bias)
    network.append(hidden_layer)
    return network

def generate_hidden_layer_matrix(dataset, bias, network):
    n = len(dataset)
    m = len(network)
    H = np.zeros((n, m))
    bias = bias[0]["bias"]
    weights = network[0:n] 
    
    for i in range(n):
        for j in range(m):
            H[i][j] = transfer(np.sum(np.dot(weights[j]["weights"], dataset[i][j] + bias[j])))
    
    return H

# Extreme learning machine algoritam.
def elm_algorithm(dataset, n_hidden_nodes, train_size):    
    n_inputs = len(dataset[0]) - 1
    n_outputs = len(set([row[-1] for row in dataset]))
    np.random.shuffle(dataset)
    splice_point = int(len(dataset)*train_size)
    train_set = dataset[0:splice_point]
    test_set = dataset[splice_point+1:len(dataset)-1]
    sum = 0
    
    # Inicijalizacija tezina i bias-a.
    network = elm_initialize(n_inputs, n_hidden_nodes, n_outputs)
    
    # Generisanje matrice skrivenog sloja.
    H = generate_hidden_layer_matrix(dataset, network[0], network[1])
    
    # Generisanje izlaza.
    T = [row[-1] for row in dataset]
    
    #hidden_layer, error, _, _ = np.linalg.lstsq(np.linalg.pinv(H).T, T)
    hidden_layer, error, _, _ = np.linalg.lstsq(H, T)
    
    for row in hidden_layer:
        sum += row
        
    accuracy = 1.0 / (1.0 + exp(-sum))
    
#     predictions = list()
#     outputs = list()
#     for test_row in test_set:
#         for hidden_row in hidden_layer:
#             output = activate(hidden_row, test_row)
#             outputs.append(output)
            
#         prediction = outputs.index(max(outputs))    
        
#         predictions.append(prediction)
    
#     actual = [row[-1] for row in test_set]
#     accuracy = calculate_accuracy(actual, predictions)
    
    return accuracy * 100

seed(1)
# Ucitavanje i formatiranje podataka.
dataset = load_csv('breast_cancer.csv')

for i in range(len(dataset[0])-1):
    str_column_to_float(dataset, i)
    
# Konvertovanje class kolone u int.
str_column_to_int(dataset, len(dataset[0])-1)

# Normalizacija.
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)

# Neuronska mreza sa propagacijom unapred.
# n_folds = 3
# learn_rate = 0.25
# n_epochs = 500
# n_hidden_nodes = 5
# ffnn = feedforward_algorithm(dataset, n_folds, learn_rate, n_epochs, n_hidden_nodes)
# print('Feedforward neural network:')
# print('Scores: %s' % ffnn)
# print('Mean Accuracy: %.3f%%' % (sum(ffnn)/float(len(ffnn))))

# Extreme learning machine.
n_hidden_nodes = 5 
train_size = 0.7
elm = elm_algorithm(dataset, n_hidden_nodes, train_size)
print('Extreme learning machine:')
print('Accuracy: %.3f%%' % elm)

Extreme learning machine:
Accuracy: 90.448%
